In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Sudoku Solver using Annealing

Napisz program poszukujący rozwiązania łamigłówki Sudoku za pomocą symulowanego
wyżarzania. 

- Plansza 9 ×9 ma zostać wczytana z pliku tekstowego, w którym pola puste
zaznaczone są znakiem x. 
- Jako funkcję kosztu przyjmij sumę powtórzeń cyfr występu-
jących w wierszach bloku 9 ×9, kolumnach bloku 9 ×9 oraz blokach 3 ×3. 
- Zaproponuj metodę generacji stanu sąsiedniego. 
- Przedstaw zależność liczby iteracji algorytmu od liczby pustych miejsc na planszy. 
- Czy Twój program jest w stanie znaleźć poprawne rozwiązanie dla każdej z testowanych konfiguracji wejściowych?

## Table of Contents

1. [Read Sudokus](#1-read-sudokus)
2. [Fill Sudoku](#2-fill-sudoku)  
    2.1 [Fill Sudoku](#21-fill-sudoku)  
    2.2 [Create sudoku](#22-create-sudoku)
3. [Annealing](#3-annealing)  
    3.1 [Cost Function](#31-cost-function)  
    3.2 [Annealing Algorithm](#32-annealing-algorithm)
4. [Vizualization](#4-vizualization)
5. [Results](#5-results)  
   5.1 [Test of hard sudokus](#51-test-of-hard-sudokus)  
   5.2 [Dependence of number of iterations on number of empty cells](#52-dependence-of-number-of-iterations-on-number-of-empty-cells)

## 1. Read Sudokus

sudokus were taken from [http://magictour.free.fr/top1465](http://magictour.free.fr/top1465)

In [3]:
def read_sudokus(filename="sudoku.txt"):
    with open(filename, "r") as f:
        sudokus = f.readlines()
    sudokus = [list(sudoku.strip()) for sudoku in sudokus]
    return np.array([np.reshape(row, (9, 9)) for row in sudokus])

## 2. Fill Sudoku

### 2.1 Fill Sudoku

Fill sudoku with all possible values randomly

In [4]:
def fill_sudoku(sudoku):
    positions = np.argwhere(sudoku == "x")
    np.random.shuffle(positions)
    numbers, counts = np.unique(sudoku, return_counts=True)
    idx = 0
    ranger = 0
    for num in [str(i) for i in range(1, 10)]:
        if num not in numbers:
            ranger = 9
        else:
            ranger = 9 - counts[np.where(numbers == num)][0]
        for _ in range(ranger):
            sudoku[positions[idx][0], positions[idx][1]] = num
            idx += 1
    return sudoku

### 2.2 Create sudoku

Create sudoku with `filled_nums` filled cells.

In [11]:
def create_sudoku(filled_nums):
    sudoku = np.array([["x" for _ in range(9)] for _ in range(9)])
    counter = {i: 0 for i in range(1, 10)}
    positions = np.argwhere(sudoku == "x")
    np.random.shuffle(positions)
    for i in range(filled_nums):
        # get random key from counter
        key = np.random.choice(list(counter.keys()))
        sudoku[positions[i][0], positions[i][1]] = str(key)
        counter[key] += 1
        if counter[key] == 9:
            del counter[key]
        
    return sudoku

array([['7', '7', '2', '9', '1', '3', '3', '5', '2'],
       ['2', '4', '1', '9', '4', '8', '6', '1', '6'],
       ['7', '4', '7', '4', '6', '8', '1', '9', '3'],
       ['5', '6', '4', '5', '5', '6', '6', '9', '9'],
       ['3', '1', '5', '8', '9', '8', '7', '6', '7'],
       ['7', '3', '5', '3', '6', '2', '4', '3', '2'],
       ['7', '9', '5', '1', '3', '9', '5', '4', '8'],
       ['1', '2', '8', '8', '4', '3', '2', '5', '9'],
       ['7', '1', '4', '2', '6', '1', '8', '8', '2']], dtype='<U1')

## 3. Annealing

### 3.1 Cost Function

In [6]:
def count_inital_energy(sudoku):
    energy = 0
    for i in range(9):
        _, cnt = np.unique(sudoku[i], return_counts=True)
        energy += np.sum(cnt-1)
        _, cnt = np.unique(sudoku[:, i], return_counts=True)
        energy += np.sum(cnt-1)
        _, cnt = np.unique(sudoku[i//3*3:i//3*3+3, i%3*3:i%3*3+3], return_counts=True)
        energy += np.sum(cnt-1)
    return energy

In [7]:
def count_swap_energy(sudoku, x1, y1, x2, y2):
    current = 0
    _, cnt = np.unique(sudoku[x1], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[x2], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[:, y1], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[:, y2], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[x1//3*3:x1//3*3+3, y1//3*3:y1//3*3+3], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[x2//3*3:x2//3*3+3, y2//3*3:y2//3*3+3], return_counts=True)
    current += np.sum(cnt-1)
    return current

### 3.2 Annealing Algorithm

In [8]:
def annealing(sudoku, temp=1, step=0.999, acceptance=0.5, max_iter=100000):
    sudoku = fill_sudoku(sudoku)
    energy = count_inital_energy(sudoku)
    data = {'iter': [], 'temp':[], 'energy': []}
    notchanged = 0

    for i in range(max_iter):
        if energy == 0:   
            sudoku, data, True
        if notchanged > 1000 or temp < 0.001:
            temp = min(1, temp+0.5)
        x1, y1, x2, y2 = np.random.randint(0, 9, 4)
        prev = count_swap_energy(sudoku, x1, y1, x2, y2)
        sudoku[x1, y1], sudoku[x2, y2] = sudoku[x2, y2], sudoku[x1, y1]
        current = count_swap_energy(sudoku, x1, y1, x2, y2)
        if current < prev or (current - prev) * np.random.rand() < acceptance * temp:
            energy += current - prev
            notchanged = 0
        else:
            sudoku[x1, y1], sudoku[x2, y2] = sudoku[x2, y2], sudoku[x1, y1]
            notchanged += 1
        temp *= step

        data['iter'].append(i)
        data['temp'].append(temp)
        data['energy'].append(energy)

    return sudoku, data, False

## 4. Vizualization

In [9]:
def plot(data):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].plot(data['iter'], data['temp'])
    ax[0].set_title('Temperature')
    ax[1].plot(data['iter'], data['energy'])
    ax[1].set_title('Energy')
    plt.show()

## 5. Results

### 5.1 Test of hard sudokus

In [12]:
def test_hard():
    sudoku = read_sudokus()
    for i in range(20):
        _, _, success = annealing(sudoku[i])
        print(f"Test {i+1}: {success}")

test_hard()

Test 1
Success: False
Test 2
Success: False
Test 3


KeyboardInterrupt: 

### 5.2 Dependence of number of iterations on number of empty cells

In [13]:
def depenence():
    data = np.zeros(82)
    for i in range(82):
        sudoku = create_sudoku(i)
        _, res, _ = annealing(sudoku)
        data[i] = res['iter'][-1]
    plt.plot(data)

depenence()

KeyboardInterrupt: 

---